<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">ANÁLISIS DEL DESPLIEGUE DE MODELOS PARA UN SISTEMA DE CLASIFICACIÓN CON IMÁGENES DE RESONANCIA MAGNÉTICA </strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [81]:
#Conectar a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
#Librería requeridas.
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import openpyxl
import joblib


#### DEFINICIÓN DE RUTAS.
---


In [83]:
#Definición de rutas
base_path = '/content/drive/MyDrive/operaciones/salidas' #Ruta del conjunto de datos.

ruta_x = '/content/drive/MyDrive/operaciones/salidas/data_final/x_val.pkl'
ruta_y = '/content/drive/MyDrive/operaciones/salidas/data_final/y_val.pkl'
ruta_modelo3 = '/content/drive/MyDrive/operaciones/operaciones/salidas/best_model/modelo_hiperparametros.keras'
ruta_modelo2 = '/content/drive/MyDrive/operaciones/operaciones/salidas/best_model/modelo_regulacion.keras'
ruta_modelo1 = '/content/drive/MyDrive/operaciones/operaciones/salidas/best_model/modelo_base.keras'
ruta_modelo4 = '/content/drive/MyDrive/operaciones/operaciones/salidas/best_model/modelo_liviano.keras'
ruta_resultado = '/content/drive/MyDrive/operaciones/salidas/despliegue/resultado.xlsx'

#### LECTURA DE LOS DATOS DE VALIDACIÓN Y ESCALAMIENTO.
---


In [84]:
# Cargar datos con joblib
x = joblib.load(ruta_x)
y = joblib.load(ruta_y)


In [85]:
# Escalamiento imágenes
x = np.array(x).astype('float32')/255.0

#### DESPLIGUE DEL MODELO.
---

In [86]:
# Cargar modelo
modelo = tf.keras.models.load_model(ruta_modelo4)

In [87]:
# Realizar predicciones
prob = modelo.predict(x)
y_pred = np.argmax(prob, axis=1)

22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step


In [88]:
# Definir nombres de clases
class_names = ['Glioma', 'Meningioma', 'No tumor', 'Pituitary']

# Convertir etiquetas verdaderas si están codificadas como índices
if isinstance(y[0], int) or isinstance(y[0], np.integer):
    y_true_labels = [class_names[i] for i in y]
else:
    y_true_labels = y

# Convertir predicciones a etiquetas
y_pred_labels = [class_names[i] for i in y_pred]

# Calcular precisión
accuracy = np.mean(np.array(y_true_labels) == np.array(y_pred_labels))
print(f"Precisión en validación: {accuracy:.2%}")

Precisión en validación: 92.07%


In [89]:
# Guardar resultados en Excel
df_resultados = pd.DataFrame({
    'paciente': [f'img_{i}' for i in range(len(x))],
    'etiqueta_real': y_true_labels,
    'clasificacion': y_pred_labels
})

df_resultados.to_excel(ruta_resultado, index=False)
print(f"Resultados guardados en: {ruta_resultado}")

Resultados guardados en: /content/drive/MyDrive/operaciones/salidas/despliegue/resultado.xlsx


#### CONCLUSIONES FINALES.
---



`El modelo básico de redes neuronales convolucionales y con regularización tienen buena precisión, ambos predicen correctamente alrededor de 91.35% de los tumores y no tumor.Por otro lado, el modelo ligero tiene una leve mejoría en la precisión con un90.07%. En contraste con el modelo de hiperparámetros que no supera más de 70% en la precisión, siendo este último el peor modelo. `